In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import pickle
from sklearn.ensemble import GradientBoostingClassifier
import json

def rlmse_score(y_test, y_hat):
    assert len(y_test) == len(y_hat)
    return np.sqrt(np.mean((np.log(1+y_hat) - np.log(1+y_test))**2))

def rlmse_scorer(estimator, X, y):
    y_hat = estimator.predict(X)
    
    return rlmse_score(y, y_hat)

In [5]:
df_test = pd.read_csv('test_3_col.csv', sep='\t', index_col='id')

In [6]:
df_test.head(5)

,Unnamed: 0,name,description
id,,,
285ea2e9935ccdeb8378c6a5,4,Стремянка,"Стремянка трехсекционная,3×4"
adfb73820bbb831257df6e95,9,Плита электрическая,Плита ЭВИ 5120. Работают 3 конфорки и духовка....
783025601c36202f633fc6a5,15,Диски R17 Rays MazdaSpeed Touring 5х114.3 7J,Все лоты привезены с площадок и аукционов Япон...
2f0cd2d2e15dc90afd847f95,19,Batman Archam Knight,NaN
5c23a37902855a20172845a5,20,Ледобур для зимней рыбалки,Отдаю ледобур времён с.с.с.р диаметр 130 мм бе...


In [7]:
df_train = pd.read_csv('train_4_col.csv', sep='\t', index_col='id')

In [8]:
df_train.head(5)

,Unnamed: 0,name,description,price
id,,,,
3edeb34cf93f490ff760af85,0,Сумка DG,NaN,199900
c98febd50dad3cc0ffc86085,1,Комплект,8-12 лет,35000
ade01e13912a46a99134cc75,2,Пальтишко демисезонное,"На девочку 1,5 г,состояние хорошее",30000
ab3e6941c11304c1519aef75,3,Attipas,"Размер-135mm, euro-22.5.Прочная, мягкая, не ск...",80000
252452a91c944a22c276d995,5,Жилет теплый,NaN,50000


In [9]:
def preprocess_data(df_input):
    df_output = df_input.copy()
    df_output = df_output.drop('description', axis = 1)
    df_output = df_output.loc[:, ~df_output.columns.str.contains('^Unnamed')]
    df_output['name'] = df_output.name.fillna('')
    df_output['name'] = df_output['name'].str.lower()
    df_output['name'] = df_output['name'].str.replace(u'[^а-яa-z -]','')
    
    return df_output

In [10]:
df_train_proc = preprocess_data(df_train)

In [11]:
df_train_proc.head()

,name,price
id,,
3edeb34cf93f490ff760af85,сумка dg,199900
c98febd50dad3cc0ffc86085,комплект,35000
ade01e13912a46a99134cc75,пальтишко демисезонное,30000
ab3e6941c11304c1519aef75,attipas,80000
252452a91c944a22c276d995,жилет теплый,50000


In [12]:
df_test_proc =preprocess_data(df_test)

In [13]:
df_test_proc.head()

,name
id,
285ea2e9935ccdeb8378c6a5,стремянка
adfb73820bbb831257df6e95,плита электрическая
783025601c36202f633fc6a5,диски r rays mazdaspeed touring х j
2f0cd2d2e15dc90afd847f95,batman archam knight
5c23a37902855a20172845a5,ледобур для зимней рыбалки


In [14]:
df_train_price = df_train.price

In [15]:
df_all_names = df_train_proc.append(df_test_proc)['name']

In [16]:
df_all_names.head()

id
3edeb34cf93f490ff760af85                  сумка dg
c98febd50dad3cc0ffc86085                  комплект
ade01e13912a46a99134cc75    пальтишко демисезонное
ab3e6941c11304c1519aef75                   attipas
252452a91c944a22c276d995              жилет теплый
Name: name, dtype: object

In [17]:
cv = CountVectorizer()
cv.fit_transform(df_all_names)


<2498415x214093 sparse matrix of type '<class 'numpy.int64'>'
	with 6239701 stored elements in Compressed Sparse Row format>

In [18]:
cv.vocabulary_

{'комнатыдушевой': 116070,
 'рефилы': 170323,
 'южном': 213198,
 'деньгиномера': 93715,
 'cтенка': 12267,
 'антенну': 67882,
 'блузу': 74529,
 'коробкасвадебная': 118193,
 'плановой': 155414,
 'оберегов': 145035,
 'контактная': 117253,
 'morethan': 37300,
 'isofix': 27326,
 'hupervenom': 25491,
 'евродисконт': 99257,
 'микки': 133860,
 'лыжные': 127912,
 'сантех': 174553,
 'телефонами': 189266,
 'пробыкомплектом': 163454,
 'htcразбито': 25310,
 'твtishiba': 188621,
 'stinol': 53122,
 'дисководы': 96314,
 'балку': 70425,
 'бушлатштаны': 78280,
 'кабинетные': 107893,
 'телескопик': 189197,
 'стану': 184017,
 'lbqt': 31356,
 'шерсьт': 207493,
 'спальнюдетскуюзал': 182205,
 'дерпазлы': 93937,
 'бензогенератор': 72630,
 'предметной': 162061,
 'крышечка': 122130,
 'cucine': 11921,
 'австрийское': 64469,
 'мамупапу': 129983,
 'тарты': 188426,
 'симвали': 177889,
 'санкта': 174501,
 'козочек': 114305,
 'ковалли': 113653,
 'зарубежных': 102877,
 'акцияоригинал': 66583,
 'колько': 115183,
 'fanc

In [19]:
train_data = cv.transform(df_train_proc.name)
test_data = cv.transform(df_test_proc.name)

In [21]:
from sklearn.linear_model import ElasticNet

In [22]:
regressor = ElasticNet(alpha=1.0, l1_ratio=0.5)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_data, df_train_price, test_size=0.33, random_state=42)
regressor.fit(X_train, y_train)

In [ ]:
pickle.dump(regressor, open('elasticnet.pckl', 'wb'))

In [ ]:
rlmse_scorer(regressor, X_test, y_test)